In [3]:
import pymongo
import pandas as pd
import numpy as np
import datetime

In [2]:
client = pymongo.MongoClient()
db = client['Tick']

In [27]:
# 讀取連續資料做回測
stk_test = pd.DataFrame(db['STK'].find({}, {'_id': 0}))
stk_test['datetime'] = pd.to_datetime(stk_test['datetime'])
stk_return = stk_test[stk_test['datetime'] >= datetime.datetime(2022,10,20,9,0,0)]
stk_return = stk_test[stk_test['datetime'] < datetime.datetime(2022,10,20,13,25,0)]
stk_test = stk_test[stk_test['datetime'] >= datetime.datetime(2022,10,19,9,0,0)]
stk_test = stk_test[stk_test['datetime'] < datetime.datetime(2022,10,19,13,25,0)]
stk_test.set_index('datetime', inplace=True)


In [30]:
total_list = list()
for code in stk_test['code'].unique():
    time_list = list()
    single_stk_list = list()
    for t in stk_test[stk_test['code'] == code].index:
        time_package = [t - datetime.timedelta(milliseconds=0.05), t + datetime.timedelta(milliseconds=0.05)]
        time_list.append(t)
        single_stk_list.append(stk_test[(stk_test.index > time_package[0]) & (stk_test.index <= time_package[1])].groupby('datetime').sum())

    # 左側外盤成交(買), 右側內盤成交(賣)
    total = [0, 0]
    iso = [0, 0]
    nis = [0, 0]
    nis_s = [0, 0]
    nis_c = [0, 0]
    nis_b = [0, 0]

    for i in range(len(single_stk_list)):
        tick_type = single_stk_list[i].at[time_list[i], 'tick_type']
        volume = int(single_stk_list[i].at[time_list[i], 'volume'])
        data = single_stk_list[i]
        if len(data) == 1:
            if tick_type == '1': # 外盤成交
                iso[0] += volume
            elif tick_type == '2': # 內盤成交
                iso[1] += volume
        else:
            if len(data['code'].unique()) == 1:
                if tick_type == '1': # 外盤成交
                    nis_s[0] += volume
                elif tick_type == '2': # 內盤成交
                    nis_s[1] += volume
            else:
                if len(data[data['code'] == '2609']) == 1:
                    if tick_type == '1': # 外盤成交
                        nis_c[0] += volume
                    elif tick_type == '2': # 內盤成交
                        nis_c[1] += volume
                else:
                    if tick_type == '1': # 外盤成交
                        nis_b[0] += volume
                    elif tick_type == '2': # 內盤成交
                        nis_b[1] += volume
    nis = np.sum([nis_s, nis_c, nis_b], axis=0).tolist()
    total = np.sum([iso, nis], axis=0).tolist()
    total_list.append(pd.DataFrame(dict(zip([(code, 'Long'), (code, 'Short')], zip(total, iso, nis, nis_s, nis_c, nis_b)))))
    print('code:{} total {}, iso {}, nis {}, nis_s {}, nis_c {}, nis_b {}'.format(code, total, iso, nis, nis_s, nis_c, nis_b))




code:2609 total [23282, 18501], iso [23091, 18194], nis [191, 307], nis_s [152, 298], nis_c [39, 9], nis_b [0, 0]
code:2912 total [284, 393], iso [270, 393], nis [14, 0], nis_s [3, 0], nis_c [0, 0], nis_b [11, 0]
code:3037 total [23996, 17949], iso [23620, 17253], nis [376, 696], nis_s [287, 687], nis_c [0, 0], nis_b [89, 9]
code:2603 total [23158, 20179], iso [23022, 19932], nis [136, 247], nis_s [26, 228], nis_c [84, 1], nis_b [26, 18]
code:2892 total [5216, 7441], iso [4980, 7434], nis [236, 7], nis_s [164, 0], nis_c [0, 0], nis_b [72, 7]
code:2330 total [13548, 23871], iso [13085, 23490], nis [463, 381], nis_s [362, 331], nis_c [2, 1], nis_b [99, 49]
code:2303 total [28385, 31103], iso [28279, 30609], nis [106, 494], nis_s [0, 389], nis_c [0, 2], nis_b [106, 103]
code:6505 total [782, 1938], iso [772, 1914], nis [10, 24], nis_s [0, 0], nis_c [0, 5], nis_b [10, 19]
code:2801 total [2467, 3509], iso [2432, 3422], nis [35, 87], nis_s [0, 74], nis_c [0, 0], nis_b [35, 13]
code:1605 tot

In [31]:
total_df = pd.concat(total_list, axis=1)
total_df.index = ['all', 'iso', 'nis', 'nis_s', 'nis_c', 'nis_b']
total_df.groupby(level=[0]).sum()

2609        2912         3037          2603         2892        ...  \
        Long  Short Long Short   Long  Short   Long  Short  Long Short  ...   
all    23282  18501  284   393  23996  17949  23158  20179  5216  7441  ...   
iso    23091  18194  270   393  23620  17253  23022  19932  4980  7434  ...   
nis      191    307   14     0    376    696    136    247   236     7  ...   
nis_b      0      0   11     0     89      9     26     18    72     7  ...   
nis_c     39      9    0     0      0      0     84      1     0     0  ...   
nis_s    152    298    3     0    287    687     26    228   164     0  ...   

      2345        1326        1301        2412        3045        
      Long Short  Long Short  Long Short  Long Short  Long Short  
all    969  1147  1005  1830  4601  3732  1618  7403  1788  2032  
iso    941  1134   995  1725  4568  3674  1606  7332  1770  2027  
nis     28    13    10   105    33    58    12    71    18     5  
nis_b    0     3    10    66    31    58    12    71    18     5  
nis_c    0     1     0     0     0     0     0     0     0     0  
nis_s   28     9     0    39     2     0     0     0     0     0  

[6 rows x 100 columns]

In [32]:
# 計算(COI_b - COI_s) / (COI_b + COI_s)
total_df.groupby(level=0, axis=1).diff(-1).dropna(axis=1).droplevel(level=1, axis=1)
score = total_df.groupby(level=0, axis=1).diff(-1).dropna(axis=1).droplevel(level=1, axis=1) / \
    total_df.groupby(level=[0], axis=1).sum()
score.fillna(0, inplace=True)

In [26]:
# 論文裡最高分的
nis_s_nis = score.rank(axis=1).loc['nis_s', :] - score.rank(axis=1).loc['nis', :]
short = nis_s_nis.sort_values()[:5]
long = nis_s_nis.sort_values()[-5:]
print(f'要Long這五檔: {long.index}')
print(f'要Short這五檔: {short.index}')

要Long這五檔: Index(['1216', '2308', '2891', '1590', '2892'], dtype='object')
要Short這五檔: Index(['2207', '4904', '2395', '5880', '1301'], dtype='object')


In [67]:
# 分成很多類來比較損益看看
# 1. 

# 損益
rt = np.log(stk_return.groupby('code')[['code', 'close']].tail(1).sort_values('code').set_index('code').astype(float) / \
        stk_return.groupby('code')[['code', 'close']].head(1).sort_values('code').set_index('code').astype(float))
coitype = score.index
for c in coitype:
        for c2 in coitype:
                if c == c2:
                        continue
                rank = score.rank(axis=1).loc[c, :] - score.rank(axis=1).loc[c2, :]
                long = rank.sort_values()[-5:]
                short = rank.sort_values()[:5]
                print('coitype: {} - {}, return: {:.4f}'.format(c, c2, (rt.loc[long.index].sum() - rt.loc[short.index].sum())["close"]))
                
                print('\n')
        for c2 in coitype:       
                rank = score.rank(axis=1).loc[c, :] + score.rank(axis=1).loc[c2, :]
                long = rank.sort_values()[-5:]
                short = rank.sort_values()[:5]
                print('coitype: {} + {}, return: {:.4f}'.format(c, c2, (rt.loc[long.index].sum() - rt.loc[short.index].sum())["close"]))        

coitype: all, return: 0.0827
coitype: iso, return: 0.0827
coitype: nis, return: 0.1399
coitype: nis_s, return: 0.1140
coitype: nis_c, return: -0.0320
coitype: nis_b, return: 0.0459


In [70]:
# 分成很多類來比較損益看看
# 1. 

# 損益
rt = np.log(stk_return.groupby('code')[['code', 'close']].tail(1).sort_values('code').set_index('code').astype(float) / \
        stk_return.groupby('code')[['code', 'close']].head(1).sort_values('code').set_index('code').astype(float))
coitype = score.index
for c in coitype:
    rank = score.rank(axis=1).loc[c, :]
    long = rank.sort_values()[-5:]
    short = rank.sort_values()[:5]
    print('coitype: {}, return: {:.4f}'.format(c, (rt.loc[long.index].sum() - rt.loc[short.index].sum())["close"]))
    for c2 in coitype:
        if c == c2:
                continue
        rank = score.rank(axis=1).loc[c, :] - score.rank(axis=1).loc[c2, :]
        long = rank.sort_values()[-5:]
        short = rank.sort_values()[:5]
        print('coitype: {} - {}, return: {:.4f}'.format(c, c2, (rt.loc[long.index].sum() - rt.loc[short.index].sum())["close"]))
            
    print('\n')
    for c2 in coitype:
        if c == c2:
                continue
        rank = score.rank(axis=1).loc[c, :] + score.rank(axis=1).loc[c2, :]
        long = rank.sort_values()[-5:]
        short = rank.sort_values()[:5]
        print('coitype: {} + {}, return: {:.4f}'.format(c, c2, (rt.loc[long.index].sum() - rt.loc[short.index].sum())["close"]))
    print('\n')

coitype: all, return: 0.0827
coitype: all - iso, return: 0.0579
coitype: all - nis, return: -0.0400
coitype: all - nis_s, return: -0.0556
coitype: all - nis_c, return: 0.1022
coitype: all - nis_b, return: 0.0368


coitype: all + iso, return: 0.0827
coitype: all + nis, return: 0.0907
coitype: all + nis_s, return: 0.1339
coitype: all + nis_c, return: 0.0133
coitype: all + nis_b, return: -0.0212


coitype: iso, return: 0.0827
coitype: iso - all, return: -0.0513
coitype: iso - nis, return: -0.0400
coitype: iso - nis_s, return: -0.0556
coitype: iso - nis_c, return: 0.1022
coitype: iso - nis_b, return: 0.0368


coitype: iso + all, return: 0.0827
coitype: iso + nis, return: 0.0907
coitype: iso + nis_s, return: 0.1339
coitype: iso + nis_c, return: 0.0133
coitype: iso + nis_b, return: -0.0212


coitype: nis, return: 0.1399
coitype: nis - all, return: 0.0400
coitype: nis - iso, return: 0.0400
coitype: nis - nis_s, return: -0.0477
coitype: nis - nis_c, return: -0.2084
coitype: nis - nis_b, return

In [50]:
rt.loc[long.index].sum() - rt.loc[short.index].sum()

close    0.082676
dtype: float64